# 03. Regresión con MLP - Soluciones
Este notebook muestra soluciones a los tres ejercicios de regresión usando PyTorch.

## Configuración
Se importan todas las librerías necesarias.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import torch
from torch.utils.data import TensorDataset, DataLoader

## Ejercicio 1
Ajustar una red MLP que consiga error cero sobre los diez datos.

In [ ]:
# Datos originales
X = np.array([258.0, 270.0, 294.0, 320.0, 342.0, 368.0, 396.0, 446.0, 480.0, 586.0])[:, None]
y = np.array([236.4, 234.4, 252.8, 298.6, 314.2, 342.2, 360.8, 368.0, 391.2, 390.8])

plt.scatter(X, y)
plt.xlabel('X')
plt.ylabel('y')
plt.show()

# Normalización
x_mean, x_std = X.mean(), X.std()
y_mean, y_std = y.mean(), y.std()
X_norm = (X - x_mean) / x_std
y_norm = (y - y_mean) / y_std

tensor_X = torch.tensor(X_norm, dtype=torch.float32)
tensor_y = torch.tensor(y_norm, dtype=torch.float32)
train_ds = TensorDataset(tensor_X, tensor_y)
train_dl = DataLoader(train_ds, shuffle=True)

class PerfectMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(1, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 20),
            torch.nn.ReLU(),
            torch.nn.Linear(20, 1)
        )
    def forward(self, x):
        return self.net(x)

model = PerfectMLP()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_history = []
for epoch in range(2000):
    for features, targets in train_dl:
        preds = model(features)
        loss = loss_fn(preds.squeeze(), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_history.append(loss.item())
    if loss.item() < 1e-6:
        break

plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

model.eval()
with torch.no_grad():
    y_hat = model(tensor_X).squeeze().numpy()
y_hat = y_hat * y_std + y_mean

plt.scatter(X, y, label='Datos')
plt.plot(X, y_hat, label='MLP', color='orange')
plt.legend()
plt.show()

## Ejercicio 2
Ajuste de un seno usando un MLP.

In [ ]:
# Generación del seno
cycles = 2
n = 100
length = 2 * np.pi * cycles
x = np.arange(0, length, length / n)
y = np.sin(x)

plt.plot(x, y, '-')
plt.show()

# Normalización
x_mean, x_std = x.mean(), x.std()
y_mean, y_std = y.mean(), y.std()
x_norm = (x - x_mean) / x_std
y_norm = (y - y_mean) / y_std

tensor_x = torch.tensor(x_norm[:, None], dtype=torch.float32)
tensor_y = torch.tensor(y_norm, dtype=torch.float32)
ds = TensorDataset(tensor_x, tensor_y)
dl = DataLoader(ds, shuffle=True)

class SineMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(1, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

model = SineMLP()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1000):
    for features, targets in dl:
        preds = model(features)
        loss = loss_fn(preds.squeeze(), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

test_x = torch.tensor(x_norm[:, None], dtype=torch.float32)
with torch.no_grad():
    pred_norm = model(test_x).squeeze().numpy()
pred = pred_norm * y_std + y_mean

plt.plot(x, y, label='seno real')
plt.plot(x, pred, label='MLP', color='orange')
plt.legend()
plt.show()

## Ejercicio 3
Regresión con datos de felicidad.

In [ ]:
# Carga de datos
data = pd.read_csv('https://raw.githubusercontent.com/mcstllns/UNIR2024/main/data-happiness.csv')
data = data.dropna()

X = data.drop('Life.Ladder', axis=1)
y = data['Life.Ladder']

# Normalización
X_mean, X_std = X.mean(), X.std()
y_mean, y_std = y.mean(), y.std()
X_norm = (X - X_mean) / X_std
y_norm = (y - y_mean) / y_std

tensor_X = torch.tensor(X_norm.values, dtype=torch.float32)
tensor_y = torch.tensor(y_norm.values, dtype=torch.float32)
ds = TensorDataset(tensor_X, tensor_y)
dl = DataLoader(ds, shuffle=True)

class HappinessMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(9, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

model = HappinessMLP()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
mse_history = []
for epoch in range(500):
    for features, targets in dl:
        preds = model(features).squeeze()
        loss = loss_fn(preds, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    mse_history.append(loss.item())

with torch.no_grad():
    preds = model(tensor_X).squeeze()
    mse = mean_squared_error(y, preds.numpy() * y_std + y_mean)
print('MSE final:', mse)